# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [180]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [181]:
np.random.seed(30)
import random as rn
rn.seed(50)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [182]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size = 30 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [183]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video:Note that here,assuming that consecutive frames will not vary much in detail, we have skipped one frame between two frames for each image.
    img_idx = list(range(1,30,2))
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list) / batch_size)# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            x = len(img_idx)
            y = 120
            z = 120
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = imresize(image,(120,120), interp="bicubic")
                    batch_data[folder,idx] = image
                    
                    R = batch_data[folder,idx,:,:,0]#normalise and feed in the image
                    R = (R - np.mean(R)) / (np.max(R) - np.min(R))
                    batch_data[folder,idx,:,:,0] = R
                    
                    G = batch_data[folder,idx,:,:,1]#normalise and feed in the image
                    G = (G - np.mean(G)) / (np.max(G) - np.min(G))
                    batch_data[folder,idx,:,:,1] = G
                    
                    B = batch_data[folder,idx,:,:,2]#normalise and feed in the image
                    B = (B - np.mean(B)) / (np.max(B) - np.min(B))
                    batch_data[folder,idx,:,:,2] = B
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        remaining_batch = (len(folder_list) - (num_batches * batch_size))
        if (len(folder_list) % batch_size) != 0:
            x = len(img_idx)
            y = 120
            z = 120
            batch_data = np.zeros((batch_size,x,y,z,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(remaining_batch): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        image = imresize(image,(120,120), interp="bicubic")
                        batch_data[folder,idx] = image
                        R = batch_data[folder,idx,:,:,0]#normalise and feed in the image
                        R = (R - np.mean(R)) / (np.max(R) - np.min(R))
                        batch_data[folder,idx,:,:,0] = R

                        G = batch_data[folder,idx,:,:,1]#normalise and feed in the image
                        G = (G - np.mean(G)) / (np.max(G) - np.min(G))
                        batch_data[folder,idx,:,:,1] = G

                        B = batch_data[folder,idx,:,:,2]#normalise and feed in the image
                        B = (B - np.mean(B)) / (np.max(B) - np.min(B))
                        batch_data[folder,idx,:,:,2] = B

                    batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [184]:
curr_dt_time = datetime.datetime.now()
train_path = './train'
val_path = './val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 150# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 150


In [185]:
input_shape = (15,120,120,3)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

## 3D-Conv Model

In [186]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2
from keras.layers import TimeDistributed
from keras.layers.wrappers import TimeDistributed

#write your model here
model = Sequential()

model.add(Conv3D(8,(3,3,3), activation='relu',padding="same", input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv3D(16,(3,3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (2,2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(32,(3,3,3), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(Conv3D(32,(3,3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (2,2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(64,(3,3,3), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (2,2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(0.1)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [187]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_36 (Conv3D)           (None, 15, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_136 (Bat (None, 15, 120, 120, 8)   32        
_________________________________________________________________
conv3d_37 (Conv3D)           (None, 13, 118, 118, 16)  3472      
_________________________________________________________________
batch_normalization_137 (Bat (None, 13, 118, 118, 16)  64        
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 6, 59, 59, 16)     0         
_________________________________________________________________
dropout_118 (Dropout)        (None, 6, 59, 59, 16)     0         
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 6, 59, 59, 32)     13856     
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [188]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [189]:
model_name = 'model_init_3D_Conv' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [190]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [191]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/150
Source path =  ./val ; batch size = 30
Source path =  ./train ; batch size = 30


c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


 3/23 [==>...........................] - ETA: 1:50 - loss: 33.9825 - categorical_accuracy: 0.1778

c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


23/23 [==============================] - 102s 4s/step - loss: 30.1510 - categorical_accuracy: 0.2203 - val_loss: 22.9327 - val_categorical_accuracy: 0.2333

Epoch 00001: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00001-30.15099-0.22029-22.93266-0.23333.h5
Epoch 2/150
23/23 [==============================] - 84s 4s/step - loss: 18.3064 - categorical_accuracy: 0.3565 - val_loss: 16.9222 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00002-18.30644-0.35652-16.92216-0.25000.h5
Epoch 3/150
23/23 [==============================] - 89s 4s/step - loss: 10.7525 - categorical_accuracy: 0.4710 - val_loss: 11.1996 - val_categorical_accuracy: 0.3083

Epoch 00003: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00003-10.75250-0.47101-11.19960-0.30833.h5
Epoch 4/150
23/23 [==============================] - 88s 4s/step - loss: 6.8454 - categorical_accuracy: 0.5275 - val_loss: 7.2609 - val_ca


Epoch 00029: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00029-1.48118-0.84058-1.97349-0.67500.h5
Epoch 30/150
23/23 [==============================] - 87s 4s/step - loss: 1.5906 - categorical_accuracy: 0.8696 - val_loss: 1.6551 - val_categorical_accuracy: 0.7083

Epoch 00030: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00030-1.59057-0.86957-1.65510-0.70833.h5
Epoch 31/150
23/23 [==============================] - 89s 4s/step - loss: 1.1677 - categorical_accuracy: 0.9087 - val_loss: 0.9117 - val_categorical_accuracy: 0.9250

Epoch 00031: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00031-1.16772-0.90870-0.91166-0.92500.h5
Epoch 32/150
23/23 [==============================] - 88s 4s/step - loss: 0.7791 - categorical_accuracy: 0.9246 - val_loss: 0.7986 - val_categorical_accuracy: 0.9333

Epoch 00032: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00032-0.77915-0.92464-0.79857-0.93333.h5
Epoch 33/150
2

23/23 [==============================] - 88s 4s/step - loss: 0.6873 - categorical_accuracy: 0.9420 - val_loss: 0.5457 - val_categorical_accuracy: 0.9833

Epoch 00058: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00058-0.68735-0.94203-0.54568-0.98333.h5
Epoch 59/150
23/23 [==============================] - 88s 4s/step - loss: 0.8027 - categorical_accuracy: 0.9029 - val_loss: 0.9889 - val_categorical_accuracy: 0.7667

Epoch 00059: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00059-0.80269-0.90290-0.98888-0.76667.h5
Epoch 60/150
23/23 [==============================] - 88s 4s/step - loss: 0.9695 - categorical_accuracy: 0.9072 - val_loss: 1.1649 - val_categorical_accuracy: 0.7333

Epoch 00060: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00060-0.96946-0.90725-1.16494-0.73333.h5
Epoch 61/150
23/23 [==============================] - 88s 4s/step - loss: 0.7439 - categorical_accuracy: 0.9391 - val_loss: 0.6980 - val_categorical_


Epoch 00086: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00086-0.20476-0.95942-0.20669-0.81667.h5
Epoch 87/150
23/23 [==============================] - 89s 4s/step - loss: 0.2947 - categorical_accuracy: 0.9536 - val_loss: 0.3461 - val_categorical_accuracy: 0.8333

Epoch 00087: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00087-0.29473-0.95362-0.34615-0.83333.h5
Epoch 88/150
23/23 [==============================] - 88s 4s/step - loss: 0.3460 - categorical_accuracy: 0.9493 - val_loss: 0.4106 - val_categorical_accuracy: 0.7917

Epoch 00088: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00088-0.34603-0.94928-0.41060-0.79167.h5
Epoch 89/150
23/23 [==============================] - 88s 4s/step - loss: 0.5719 - categorical_accuracy: 0.9391 - val_loss: 0.8805 - val_categorical_accuracy: 0.7417

Epoch 00089: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00089-0.57193-0.93913-0.88052-0.74167.h5
Epoch 90/150
2

23/23 [==============================] - 87s 4s/step - loss: 0.3087 - categorical_accuracy: 0.9551 - val_loss: 0.3509 - val_categorical_accuracy: 0.7917

Epoch 00115: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00115-0.30873-0.95507-0.35088-0.79167.h5
Epoch 116/150
23/23 [==============================] - 88s 4s/step - loss: 0.4906 - categorical_accuracy: 0.9725 - val_loss: 0.6747 - val_categorical_accuracy: 0.8000

Epoch 00116: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00116-0.49062-0.97246-0.67466-0.80000.h5
Epoch 117/150
23/23 [==============================] - 88s 4s/step - loss: 0.5797 - categorical_accuracy: 0.9536 - val_loss: 0.5156 - val_categorical_accuracy: 0.8083

Epoch 00117: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00117-0.57973-0.95362-0.51564-0.80833.h5
Epoch 118/150
23/23 [==============================] - 89s 4s/step - loss: 0.4547 - categorical_accuracy: 0.9565 - val_loss: 0.6227 - val_categoric


Epoch 00143: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00143-0.45108-0.98261-0.73519-0.91667.h5
Epoch 144/150
23/23 [==============================] - 88s 4s/step - loss: 0.3923 - categorical_accuracy: 0.9812 - val_loss: 0.3958 - val_categorical_accuracy: 0.9417

Epoch 00144: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00144-0.39232-0.98116-0.39578-0.94167.h5
Epoch 145/150
23/23 [==============================] - 90s 4s/step - loss: 0.2294 - categorical_accuracy: 0.9667 - val_loss: 0.2743 - val_categorical_accuracy: 0.8000

Epoch 00145: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00145-0.22940-0.96667-0.27434-0.80000.h5
Epoch 146/150
23/23 [==============================] - 87s 4s/step - loss: 0.1471 - categorical_accuracy: 0.9884 - val_loss: 0.2073 - val_categorical_accuracy: 0.9583

Epoch 00146: saving model to model_init_3D_Conv_2019-03-1518_12_18.422945/model-00146-0.14710-0.98841-0.20733-0.95833.h5
Epoch 147/1

### Got best Accuracy at epoch: 147
#### Train Accuracy: 0.9725
#### Validation Accuracy: 0.9833

## CNN - RNN Model

In [206]:
model = Sequential()

model.add(TimeDistributed(Conv2D(8, (3, 3), padding='same'), input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(8, (3, 3))))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(Dropout(0.25))

model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same')))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(32, (3, 3))))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(Dropout(0.25))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(128, kernel_regularizer=l2(0.1))))
model.add(Dropout(0.25))
model.add(GRU(output_dim=128, return_sequences=True, kernel_regularizer=l2(0.1)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(return_sequences=True, kernel_regularizer=<keras.reg..., units=128)`


### Repeating steps executed for 3D-Conv Model

In [207]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_231 (TimeDi (None, 15, 120, 120, 8)   224       
_________________________________________________________________
activation_132 (Activation)  (None, 15, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization_156 (Bat (None, 15, 120, 120, 8)   32        
_________________________________________________________________
time_distributed_232 (TimeDi (None, 15, 118, 118, 8)   584       
_________________________________________________________________
activation_133 (Activation)  (None, 15, 118, 118, 8)   0         
_________________________________________________________________
batch_normalization_157 (Bat (None, 15, 118, 118, 8)   32        
_________________________________________________________________
time_distributed_233 (TimeDi (None, 15, 59, 59, 8)     0         
__________

In [208]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [209]:
model_name = 'model_init_CNN_RNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [210]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [211]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/150
Source path =  Source path =  ./train ; batch size =./val 30 ; batch size = 30



c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


 3/23 [==>...........................] - ETA: 2:16 - loss: 46.4578 - categorical_accuracy: 0.1222

c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\wbt-gamelauncher\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


23/23 [==============================] - 122s 5s/step - loss: 36.4483 - categorical_accuracy: 0.2391 - val_loss: 24.1902 - val_categorical_accuracy: 0.3583

Epoch 00001: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00001-36.44831-0.23913-24.19020-0.35833.h5
Epoch 2/150
23/23 [==============================] - 95s 4s/step - loss: 17.5753 - categorical_accuracy: 0.3551 - val_loss: 11.9192 - val_categorical_accuracy: 0.3667

Epoch 00002: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00002-17.57525-0.35507-11.91921-0.36667.h5
Epoch 3/150
23/23 [==============================] - 103s 4s/step - loss: 9.7550 - categorical_accuracy: 0.4971 - val_loss: 8.0568 - val_categorical_accuracy: 0.4833

Epoch 00003: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00003-9.75500-0.49710-8.05677-0.48333.h5
Epoch 4/150
23/23 [==============================] - 101s 4s/step - loss: 6.9321 - categorical_accuracy: 0.6203 - val_loss: 6.5451 - val_cate


Epoch 00029: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00029-0.43040-0.93623-0.54554-0.75000.h5
Epoch 30/150
23/23 [==============================] - 106s 5s/step - loss: 0.4894 - categorical_accuracy: 0.9217 - val_loss: 0.8018 - val_categorical_accuracy: 0.7167

Epoch 00030: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00030-0.48942-0.92174-0.80179-0.71667.h5
Epoch 31/150
23/23 [==============================] - 105s 5s/step - loss: 0.5054 - categorical_accuracy: 0.9246 - val_loss: 0.7130 - val_categorical_accuracy: 0.7250

Epoch 00031: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00031-0.50541-0.92464-0.71298-0.72500.h5
Epoch 32/150
23/23 [==============================] - 106s 5s/step - loss: 0.6016 - categorical_accuracy: 0.9290 - val_loss: 1.2018 - val_categorical_accuracy: 0.6417

Epoch 00032: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00032-0.60159-0.92899-1.20180-0.64167.h5
Epoch 33/15

23/23 [==============================] - 103s 4s/step - loss: 0.9996 - categorical_accuracy: 0.9246 - val_loss: 1.1175 - val_categorical_accuracy: 0.7583

Epoch 00058: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00058-0.99957-0.92464-1.11754-0.75833.h5
Epoch 59/150
23/23 [==============================] - 103s 4s/step - loss: 0.9179 - categorical_accuracy: 0.9290 - val_loss: 0.9971 - val_categorical_accuracy: 0.7417

Epoch 00059: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00059-0.91793-0.92899-0.99714-0.74167.h5
Epoch 60/150
23/23 [==============================] - 103s 4s/step - loss: 0.8722 - categorical_accuracy: 0.9246 - val_loss: 1.3949 - val_categorical_accuracy: 0.7000

Epoch 00060: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00060-0.87222-0.92464-1.39494-0.70000.h5
Epoch 61/150
23/23 [==============================] - 103s 4s/step - loss: 0.9929 - categorical_accuracy: 0.9130 - val_loss: 1.0836 - val_categori


Epoch 00086: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00086-0.23007-0.95797-0.55661-0.76667.h5
Epoch 87/150
23/23 [==============================] - 104s 5s/step - loss: 0.5949 - categorical_accuracy: 0.9319 - val_loss: 1.5386 - val_categorical_accuracy: 0.6583

Epoch 00087: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00087-0.59486-0.93188-1.53865-0.65833.h5
Epoch 88/150
23/23 [==============================] - 102s 4s/step - loss: 1.6806 - categorical_accuracy: 0.9087 - val_loss: 2.3724 - val_categorical_accuracy: 0.6750

Epoch 00088: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00088-1.68059-0.90870-2.37245-0.67500.h5
Epoch 89/150
23/23 [==============================] - 105s 5s/step - loss: 2.1963 - categorical_accuracy: 0.9145 - val_loss: 2.2568 - val_categorical_accuracy: 0.7333

Epoch 00089: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00089-2.19632-0.91449-2.25675-0.73333.h5
Epoch 90/15

Epoch 115/150
23/23 [==============================] - 102s 4s/step - loss: 0.0966 - categorical_accuracy: 0.9594 - val_loss: 0.1978 - val_categorical_accuracy: 0.7833

Epoch 00115: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00115-0.09664-0.95942-0.19780-0.78333.h5
Epoch 116/150
23/23 [==============================] - 104s 5s/step - loss: 0.1139 - categorical_accuracy: 0.9609 - val_loss: 0.2692 - val_categorical_accuracy: 0.7833

Epoch 00116: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00116-0.11387-0.96087-0.26923-0.78333.h5
Epoch 117/150
23/23 [==============================] - 105s 5s/step - loss: 0.0997 - categorical_accuracy: 0.9609 - val_loss: 0.2023 - val_categorical_accuracy: 0.8000

Epoch 00117: saving model to model_init_CNN_RNN_2019-03-1518_12_18.422945/model-00117-0.09972-0.96087-0.20233-0.80000.h5
Epoch 118/150
23/23 [==============================] - 104s 5s/step - loss: 0.0808 - categorical_accuracy: 0.9739 - val_loss: 0.16

### Best Accuracy at epoch: 120
#### Train Acc: 9609
#### Validation Acc: 9750

Thus, from the results obtained above we conclude the 3D Convolutional model to be the best and final model, as it gives the highest accuracy score and has fewer number of model parameters.